### Phi-2 (FP16)
#### NOTE: YOU WILL NEED LLAMA-CPP-PYTHON VERSION v0.2.24 OR HIGHER

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation in Linux and with Nvidia's CUDA.

See the [README.md](README.md) file for help on how to run this.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

/home/mark/miniconda3/envs/LlamaIndexRAGLinux/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/phi-2-ggml-model-f16.gguf', # FP16 version of Phi-2
    temperature=0.1,
    max_new_tokens=1024,
    context_window=2048, # Phi-2 2K context window - this could be a limitation for RAG as it has to put the content into this context window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 33}, # 33 layers were all that were required and this fit within the RTX 3090's 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3090, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 18 key-value pairs and 325 tensors from ./Models/phi-2-ggml-model-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count 

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", cache_folder=None)

#### 6. Prompt Template

Prompt template for Phi-2 is below. As there's only a prompt we will combine the system message and prompt into the prompt.

Instruct: {prompt}<br>
Output:

In [5]:
# Produces a prompt specific to the model
def modelspecific_prompt(promptmessage):
    # As per https://huggingface.co/TheBloke/phi-2-GGUF
    return f"Instruct: {promptmessage}\nOutput:"

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256 # Number of tokens in each chunk

#### 8. Index documents

In [7]:
index = VectorStoreIndex.from_documents(documents)

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [8]:
from llama_index.core.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    # citation_chunk_size=256 #256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

In [10]:
# Let's look at the chunking and prompting parameters from our service_context

print("service_context.node_parser:")
print("Chunk Overlap:",Settings.node_parser.chunk_overlap)
print("Chunk Size:",Settings.node_parser.chunk_size)
print("Paragraph Separator:",Settings.node_parser.paragraph_separator.replace("\n","\\n"))
print("Secondary Chunking RegEx:",Settings.node_parser.secondary_chunking_regex)
print("Include Metadata:",Settings.node_parser.include_metadata)

print("\nservice_context.prompt_helper:")
print("Context Window:", Settings.prompt_helper.context_window) # The maximum context size that will get sent to the LLM.
print("Chunk Overlap Ratio:", Settings.prompt_helper.chunk_overlap_ratio) # The percentage token amount that each chunk should overlap.
print("Chunk Size Limit:", Settings.prompt_helper.chunk_size_limit) # The maximum size of a chunk.
print("Chunk Separator: '", Settings.prompt_helper.separator, "'") # The separator when chunking tokens.
print("Num Output: '", Settings.prompt_helper.num_output, "'") # The number of tokens that should be left to generate in the prompt (default is 256)

service_context.node_parser:
Chunk Overlap: 200
Chunk Size: 256
Paragraph Separator: \n\n\n
Secondary Chunking RegEx: [^,.;。？！]+[,.;。？！]?
Include Metadata: True

service_context.prompt_helper:
Context Window: 2048
Chunk Overlap Ratio: 0.1
Chunk Size Limit: None
Chunk Separator: '   '
Num Output: ' 1024 '


#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is set here, adjust it to match what you want the LLM to act like and do.

In [11]:
def RunQuestion(questionText):
    # Excluding the system prompt as the model is including it (even a short version of it) is causing lack of responses in some cases and it is not consistently answering.
    prompt = "" # "You are a story teller who likes to elaborate and answers questions in a positive, helpful and interesting way, so please answer the following question - "
    
    prompt = prompt + questionText

    queryQuestion = modelspecific_prompt(prompt)

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [12]:
TestQuestions = [
    "Summarise this story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [13]:
qa_pairs = []

for indexNum, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{indexNum}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for indexNumCit, node in enumerate(response.source_nodes, start=1):
        print(f"Source {indexNumCit} of {len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise this story for me



llama_print_timings:        load time =      66.99 ms
llama_print_timings:      sample time =      81.91 ms /   195 runs   (    0.42 ms per token,  2380.55 tokens per second)
llama_print_timings: prompt eval time =     150.01 ms /  1008 tokens (    0.15 ms per token,  6719.60 tokens per second)
llama_print_timings:        eval time =    2133.92 ms /   194 runs   (   11.00 ms per token,    90.91 tokens per second)
llama_print_timings:       total time =    2891.29 ms /  1202 tokens
Llama.generate: prefix-match hit


Source 1 of 3: |Thundertooth Part 1.docx| Source 1:
Thundertooth faced a dilemma – he was hungry, but he couldn't bring himself to feast on the humans who scurried around like ants. As his hunger grew, he stumbled upon a park, an oasis of greenery amidst the concrete and steel. The park was adorned with holographic flowers that emitted a gentle glow, creating an ethereal atmosphere.



While Thundertooth marveled at the beauty of the park, the mayor of the city happened to be passing by. Mayor Eleanor Grace, a charismatic and forward-thinking leader, was immediately intrigued by the sight of the talking dinosaur. She approached Thundertooth with a mix of curiosity and caution.



"Hello there, majestic creature. What brings you to our time?" Mayor Grace inquired, her voice calm and reassuring.



Thundertooth, though initially startled, found comfort in the mayor's soothing tone. In broken sentences, he explained his journey through time, the strange portal, and his hunger dilemma. May


llama_print_timings:        load time =      66.99 ms
llama_print_timings:      sample time =      35.07 ms /    84 runs   (    0.42 ms per token,  2395.00 tokens per second)
llama_print_timings: prompt eval time =     137.19 ms /   824 tokens (    0.17 ms per token,  6006.40 tokens per second)
llama_print_timings:        eval time =     914.64 ms /    83 runs   (   11.02 ms per token,    90.75 tokens per second)
llama_print_timings:       total time =    1306.00 ms /   907 tokens
Llama.generate: prefix-match hit


Source 1 of 3: |Thundertooth Part 3.docx| Source 1:
2. **Echo**: With his extraordinary mimicry abilities, Echo would amplify the emergency signals, ensuring that every citizen received timely warnings and instructions for evacuation.



3. **Sapphire**: Harnessing her calming and healing powers, Sapphire would assist in calming the panicked masses, ensuring an orderly and efficient evacuation.



4. **Ignis**: Drawing upon his fiery talents, Ignis would create controlled bursts of heat, attempting to alter the meteor's trajectory and reduce its destructive force.



As the citizens evacuated to designated shelters, the Thundertooth family sprang into action. Lumina worked tirelessly to strengthen the city's energy systems, Echo echoed evacuation orders through the city's speakers, Sapphire offered comfort to those in distress, and Ignis unleashed controlled bursts of flames towards the approaching meteor.



Thundertooth stood at the forefront, using his mighty roar to coordinate and 


llama_print_timings:        load time =      66.99 ms
llama_print_timings:      sample time =      14.21 ms /    34 runs   (    0.42 ms per token,  2393.52 tokens per second)
llama_print_timings: prompt eval time =     120.24 ms /   758 tokens (    0.16 ms per token,  6303.90 tokens per second)
llama_print_timings:        eval time =     362.38 ms /    33 runs   (   10.98 ms per token,    91.07 tokens per second)
llama_print_timings:       total time =     587.09 ms /   791 tokens
Llama.generate: prefix-match hit


Source 1 of 3: |Thundertooth Part 2.docx| Source 1:
Thundertooth's toy factory became a sensation, and its creations were highly sought after. The widgets incorporated cutting-edge holographic displays, levitation technology, and even the ability to change shapes and colors with a mere thought. Children across the city rejoiced as they played with these incredible toys that seemed to bring their wildest fantasies to life.



As the years passed, Thundertooth's life took a heartwarming turn. He met a kind and intelligent dinosaur named Seraphina, and together they started a family. Thundertooth and Seraphina were blessed with four children, each with unique characteristics that mirrored the diversity of their modern world.



Lumina: The eldest of Thundertooth's children, Lumina inherited her mother's intelligence and her father's sense of wonder. With sparkling scales that emitted a soft glow, Lumina had the ability to generate light at will. She became fascinated with technology, ofte


llama_print_timings:        load time =      66.99 ms
llama_print_timings:      sample time =     432.67 ms /  1024 runs   (    0.42 ms per token,  2366.69 tokens per second)
llama_print_timings: prompt eval time =     128.63 ms /   806 tokens (    0.16 ms per token,  6266.03 tokens per second)
llama_print_timings:        eval time =   11790.75 ms /  1023 runs   (   11.53 ms per token,    86.76 tokens per second)
llama_print_timings:       total time =   15739.99 ms /  1829 tokens


Source 1 of 3: |Thundertooth Part 3.docx| Source 1:
As Ignis's controlled bursts of flames interacted with the meteor, it began to change course. The combined efforts of the Thundertooth family, guided by their unique talents, diverted the catastrophic collision. The meteor, once destined for destruction, now harmlessly sailed past the Earth, leaving the city and its inhabitants unscathed.



The citizens, emerging from their shelters, erupted into cheers of gratitude. Mayor Grace approached Thundertooth, expressing her heartfelt thanks for the family's heroic efforts. The Thundertooth family, tired but triumphant, basked in the relief of having saved their beloved city from imminent disaster.



In the wake of the crisis, the citizens of the futuristic city hailed Thundertooth and his family as true heroes. The toy factory that once brought joy to children now became a symbol of resilience and unity. The Thundertooth family's legacy was forever etched in the city's history, a testamen

#### 13. Output responses

In [14]:
for indexQA, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{indexQA}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise this story for me

The story is about Thundertooth, a talking dinosaur who finds himself in the future city of Thunderville. He faces a dilemma of hunger but is intrigued by the futuristic city's beauty. The mayor of Thunderville, Eleanor Grace, approaches Thundertooth with curiosity and caution. Thundertooth explains his journey through time and his hunger dilemma to the mayor. The mayor is amazed by Thundertooth's tale and offers her assistance. Thundertooth's family, consisting of Lumina, Echo, Sapphire, Ignis, and their children, work together to protect the city from an approaching meteor. Thundertooth finds purpose in the futuristic city and starts a toy factory that produces amazing widgets. The widgets become highly sought after and bring joy to children across the city. Thundertooth meets Seraphina and starts a family with four children who possess unique characteristics. The story ends with Thundertooth's heartwarming family life in the modern world. [/INST]

--